In [6]:
from pandas import DataFrame
import sqlalchemy
from waterquality import classes
from waterquality import utils

In [7]:
inp = [{'chl':10, 'chl_corrected':100}, {'chl':11,'chl_corrected':110}, {'chl':12,'chl_corrected':120}]
df = DataFrame(inp)
print df

   chl  chl_corrected
0   10            100
1   11            110
2   12            120


In [8]:
for row in df.itertuples():
    print row    

Pandas(Index=0, chl=10, chl_corrected=100)
Pandas(Index=1, chl=11, chl_corrected=110)
Pandas(Index=2, chl=12, chl_corrected=120)


In [17]:
for row in df.itertuples():
    print "{}, {}, {}".format(row.Index, row.chl, row.chl_corrected)

['Index', 'chl', 'chl_corrected']
0, 10, 100
['Index', 'chl', 'chl_corrected']
1, 11, 110
['Index', 'chl', 'chl_corrected']
2, 12, 120


In [55]:
reload(classes)

<module 'waterquality.classes' from 'waterquality\classes.pyc'>

In [56]:
utils.recreate_tables()
classes.connect_db(classes.db_location)
session = classes.db_session(classes.db_engine)

Creating tables


In [57]:
for row in df.itertuples():
    wq = classes.WaterQuality()
    for key in vars(row).keys():  # converts named_tuple to a Dict-like and gets the keys
        if key == "Index":
            continue
        setattr(wq, key, getattr(row,key))
    session.add(wq)
    

In [58]:
session.commit()

In [1]:
import sqlalchemy
from sqlalchemy import orm
from sqlalchemy import Column, Integer, String, Float, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

db_engine = None
Base = declarative_base()
Session = None

def connect_db(database=r"C:\Users\dsx.AD3\Projects\test.sqlite"):
	"""
		Just a helper function that sets up the database engine
	:return:
	"""
	global db_engine
	db_engine = sqlalchemy.create_engine('sqlite:///{}'.format(database))


def db_session(engine=db_engine):
	"""
		provides a Session to the database - can be run for each thing that needs a new session.
	:param engine:
	:return:
	"""
	global Session

	if not Session:
		Session = orm.sessionmaker(bind=engine)

	return Session()


class Site(Base):
	__tablename__ = "sites"

	id = Column(Integer, primary_key=True)
	name = Column(String)


class WaterQuality(Base):
	"""
		Each instance of this class is an observation in the database
	"""
	__tablename__ = 'water_quality'

	id = Column(Integer, primary_key=True)

	site_id = Column(Integer, ForeignKey('sites.id'))
	site = relationship("Site",
						primaryjoin=(site_id == Site.id),
						backref="water_quality_records")

In [3]:
connect_db(r"C:\Users\dsx.AD3\Projects\test.sqlite")
Base.metadata.create_all(db_engine)

In [5]:
session = db_session()
site = Site(name="test")

'1.1.2'

'C:\\Python27\\ArcGIS10.3\\python.exe'

In [5]:
import os
import arcpy
import numpy
import pandas

outFC = os.path.join(os.getcwd(), "test.shp")

# Create a numpy array with an id field, and a field with a tuple 
#  of x,y coordinates
#

array = numpy.array([(1, -116.809, 34.271),
                     (2, -114.241, 33.684)],
                    numpy.dtype([('idfield',numpy.int32),('x_coord', '<f8'), ('y_coord', '<f8')]))

df = pandas.DataFrame(array)

# Define a spatial reference for the output feature class
#
SR = arcpy.SpatialReference(4269)

# Export the numpy array to a feature class using the XY field to
#  represent the output point feature
#
arcpy.da.NumPyArrayToFeatureClass(df.as_matrix(), outFC, ['x_coord', 'y_coord'], SR)


ImportError: No module named pandas